# Considerações iniciais
Conforme feito nos notebooks anterios, nós preparamos os dados, geramos diversidades, e selecionamos as melhores combinações de modelos. Agora iremos apenas ajustar os modelos e persistir na memória.

# Importações

In [1]:
import pandas as pd 
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import pickle as pkl

# Leitura e separação das bases em X e Y

In [2]:
train_lvl0 = pd.read_csv('train_lvl0.csv')
train_lvl1 = pd.read_csv('train_lvl1.csv')
val = pd.read_csv("valid.csv")

X_train0, Y_train0 = train_lvl0.drop(columns=['Date','Sales']), train_lvl0['Sales']
X_train1, Y_train1 = train_lvl1.drop(columns=['Date','Sales']), train_lvl1['Sales']
X_val, Y_val = val.drop(columns=['Date','Sales']), val['Sales']

preds_lvl1 = pd.DataFrame()
preds_val = pd.DataFrame()

# Criando uma lista com os modelos
Copiando os parâmetros dos modelos selecionados no notebook anterior, criamos uma lista com cada modelo, para assim facilitar na implementação.

In [3]:
models = [
        LGBMRegressor(num_leaves=110,
                        min_data_in_leaf=14,
                        learning_rate=0.03248168667854209, 
                        n_estimators=215,
                        random_state=0),
    
        LGBMRegressor(num_leaves=361,
                        min_data_in_leaf=59,
                        learning_rate=0.05378358562195618, 
                        n_estimators=276,
                        random_state=0),

        RandomForestRegressor(n_estimators=176,
                                       max_depth=3,
                                       min_samples_leaf=48,
                                       n_jobs=-1),
    
        LGBMRegressor(num_leaves=24,
                        min_data_in_leaf=1,
                        learning_rate=0.03199227519485914, 
                        n_estimators=200,
                        random_state=0),
    
        LGBMRegressor(num_leaves=185,
                        min_data_in_leaf=96,
                        learning_rate=0.014121042963223253, 
                        n_estimators=287,
                        random_state=0),

        SVR(kernel='rbf',
        C=0.6614584754426877),

        SVR(kernel='rbf',
        C=8.713863857748523),
    
        LGBMRegressor(num_leaves=238,
                        min_data_in_leaf=80,
                        learning_rate=0.05209570020716538, 
                        n_estimators=268,
                        random_state=0),
    
        LGBMRegressor(num_leaves=427,
                        min_data_in_leaf=1,
                        learning_rate=0.0001, 
                        n_estimators=236,
                        random_state=0),
    
        RandomForestRegressor(n_estimators=139,
                                   max_depth=17,
                                   min_samples_leaf=34,
                                   n_jobs=-1),
        LGBMRegressor(num_leaves=55,
                    min_data_in_leaf=48,
                    learning_rate=0.018714601098343324, 
                    n_estimators=274,
                    random_state=0),
    
      
]

# Treinamento dos modelos 
Utilizando umá estrutura de repetição, treinamos cada modelo com base em sua posição na lista.

In [4]:
for i in range(len(models)):
    models[i].fit(X_train0, Y_train0)

# Criando as previsões de primeiro nivel e da validação 
Novamente, percorrendo com uma estrutura de repetição, com base na posição de cada modelo na lista, faço a previsões de cada modelo, e os guardo em dataframes.

In [5]:
for i in range(len(models)):
    prev = models[i].predict(X_train1)
    preds_lvl1['Modelo_{}'.format(i)] = prev

    prev = models[i].predict(X_val)
    preds_val['Modelo_{}'.format(i)] = prev

# Treinando a ridge de 2º nível

Conforme feito no notebook anterior, utilizamos uma ridge no nosso segundo nível, que irá utilizar as previsões dos modelos acima como features.

In [6]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(preds_lvl1,Y_train1)
preds = ridge.predict(preds_val)

In [7]:
np.sqrt(mean_squared_error(Y_val,preds))

385.98652981196255

# Resultado validação

Conforme vemos, nossa baseline no primeiro nobeook para a validação era cerca de 3866, que eram as vendas utilizadas no dia anterior. E apesar de todo trabalho para chegar até aqui, conseguimos a marca de 385.

# Visualizando as previsões da ridge

In [8]:
preds[0:100]

array([ 5716.12732818,  6570.02509147, 10433.63918445, 11727.46530209,
        6671.86054738,  6675.36377111, 12331.55182077,  8121.65913626,
        9814.75402386,  7152.67584754,  8648.77276758,  9632.80598566,
        5291.70927796,  7016.89290764,  7446.96659215,  9832.0580813 ,
        9014.23640848,  7743.35808883,  8807.28857458, 10132.21079557,
        5842.35513923,  6832.95705613,  9857.41420906, 10806.29990296,
        6099.87289615,  6094.03756252,  9617.84749769,  7127.52382161,
        9686.33752851,  6452.61366789,  9352.81101154,  9710.80865792,
        6248.08804743,  7586.19155834,  7989.13964037,  9954.74646691,
        7824.91690016,  7854.65496018,  9673.98846101, 10386.9469018 ,
          32.00237652,  5929.0089488 ,   150.23750718, 10351.41011626,
        5634.42297906,  5661.34877867,  9816.59050748,  8991.84181069,
          21.60475997,  6282.19409116,  8810.22986694,  8674.87227977,
          40.6944794 ,    31.0558333 ,  7717.44898145,    52.71631976,
      

# Previsões na base de teste

Como essa é uma base retirada e adaptada de uma competição do kaggle, temos uma base de teste, porém os resultados não são 100% aplicáveis, pois reduzimos o número de lojas. Se baseando nesse contexto, conforme feito no notebook 1, faremos a previsão para apenas as 20 primeiras lojas.

In [9]:
test = pd.read_csv('teste.csv')
Sub = pd.DataFrame()
Sub['Id'] = test['Id']
test = test.drop(columns=['Date','Id'])

preds_test = pd.DataFrame()

for i in range(len(models)):
    prev = models[i].predict(test)
    preds_test['Modelo_{}'.format(i)] = prev

p = ridge.predict(preds_test)
p[p < 0] = 0
Sub['Sales'] = p.round().astype(int)
Sub

,Id,Sales
0,1,0
1,2,133
2,3,14
3,4,0
4,5,5
...,...,...
667,40242,0
668,40243,0
669,40244,11
670,40245,15


# Persistindo na memória os modelos para utilizações em caso de deploy

Essa é uma parte muito importante em caso de deploy, pois precisamos salvar os modelos para que possam ser utilizados no futuro em produção.

In [10]:
file_name = "Modelos.pkl"

open_file = open(file_name, "wb")
pkl.dump([models,ridge], open_file)
open_file.close()

# Leitura dos modelos para certeza de que não houve problemas

In [11]:
open_file = open(file_name, "rb")
models,ridge = pkl.load(open_file)

print(models,ridge)

[LGBMRegressor(learning_rate=0.03248168667854209, min_data_in_leaf=14,
              n_estimators=215, num_leaves=110, random_state=0), LGBMRegressor(learning_rate=0.05378358562195618, min_data_in_leaf=59,
              n_estimators=276, num_leaves=361, random_state=0), RandomForestRegressor(max_depth=3, min_samples_leaf=48, n_estimators=176,
                      n_jobs=-1), LGBMRegressor(learning_rate=0.03199227519485914, min_data_in_leaf=1,
              n_estimators=200, num_leaves=24, random_state=0), LGBMRegressor(learning_rate=0.014121042963223253, min_data_in_leaf=96,
              n_estimators=287, num_leaves=185, random_state=0), SVR(C=0.6614584754426877), SVR(C=8.713863857748523), LGBMRegressor(learning_rate=0.05209570020716538, min_data_in_leaf=80,
              n_estimators=268, num_leaves=238, random_state=0), LGBMRegressor(learning_rate=0.0001, min_data_in_leaf=1, n_estimators=236,
              num_leaves=427, random_state=0), RandomForestRegressor(max_depth=17, min_sam

# Considerações finais 

Estou utilizando esses notebooks como forma de demonstrar minhas habilidades que venho adquirindo ao longo dessa minha trajetória como estudante de engenharia de computação e cientista de dados.

Agradeço a você que tenha acompanhado esse projeto até aqui. Como ideia para futuros projetos pode ser interessante recriar com todas as lojas disponíveis no dataset original e não apenas 20.